# GJ Project: Training the Siamese Network

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from scipy.spatial import distance
import torch.nn as nn
import torchvision.datasets as dsets
from torch.autograd import Variable
import gjnn.model
import gjnn.loss
import gjnn.dataloader


# Dataset Loading 
dataset = pd.read_csv("data/ds_super_short.csv", sep=None, engine='python',  dtype={'user_id_1': "category", "user_id_2":"category"})


dataset.drop(["ifp_id"], axis =1, inplace = True)


In [2]:
dataset.head()

# To modify when dataset column order will change
features_user_1 = [0,1,2,3,4,5,6,7,8,9,10,11,15]
features_user_2 = [0,1,2,3,4,5,16,17,18,19,20,21,22]
print(dataset.columns[[0,1,2,3,4,5,6,7,8,9,10,11,15]])
print(dataset.columns[[0,1,2,3,4,5,16,17,18,19,20,21,22]])

print(dataset.iloc[:, features_user_1])
print(dataset.iloc[:, features_user_2])
user_1 = dataset.iloc[:, features_user_1]
user_2 = dataset.iloc[:, features_user_2]

print(len(dataset))

Index(['topic_0', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5',
       'expertise_1', 'user_id_1', 'value_a_1', 'value_b_1', 'value_c_1',
       'days_from_start_1', 'distance_1'],
      dtype='object')
Index(['topic_0', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5',
       'expertise_2', 'user_id_2', 'value_a_2', 'value_b_2', 'value_c_2',
       'days_from_start_2', 'distance_2'],
      dtype='object')
      topic_0  topic_1   topic_2   topic_3  topic_4  topic_5  expertise_1  \
0    0.175113      0.0  0.676763  0.143534      0.0      0.0            2   
1    0.175113      0.0  0.676763  0.143534      0.0      0.0            2   
2    0.175113      0.0  0.676763  0.143534      0.0      0.0            2   
3    0.175113      0.0  0.676763  0.143534      0.0      0.0            2   
4    0.175113      0.0  0.676763  0.143534      0.0      0.0            2   
5    0.175113      0.0  0.676763  0.143534      0.0      0.0            2   
6    0.175113      0.0  0.676763  0.1

In [3]:
user_1_dist = user_1["distance_1"]
user_1_dist

0      0.141421
1      0.141421
2      0.141421
3      0.141421
4      0.141421
5      0.141421
6      0.141421
7      0.141421
8      0.141421
9      0.141421
10     0.141421
11     0.141421
12     0.141421
13     0.141421
14     0.141421
15     0.141421
16     0.141421
17     0.141421
18     0.141421
19     0.141421
20     0.141421
21     0.141421
22     0.141421
23     0.141421
24     0.141421
25     0.141421
26     0.141421
27     0.141421
28     0.141421
29     0.141421
         ...   
970    0.141421
971    0.141421
972    0.141421
973    0.141421
974    0.141421
975    0.141421
976    0.141421
977    0.141421
978    0.141421
979    0.141421
980    0.141421
981    0.141421
982    0.141421
983    0.141421
984    0.141421
985    0.141421
986    0.141421
987    0.141421
988    0.141421
989    0.141421
990    0.141421
991    0.141421
992    0.141421
993    0.141421
994    0.141421
995    0.141421
996    0.141421
997    0.141421
998    0.141421
999    0.141421
Name: distance_1, Length

In [4]:
dataset = dataset.apply(pd.to_numeric)



# The current split is 95% of data is used for training and 5% for validation of the model
train = dataset.sample(frac=0.95,random_state=200)
test = dataset.drop(train.index)
#train = train.as_matrix()
#test = test.as_matrix()






#train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=True)
#test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=False)

# Option 1: Loading the dataset, we have no output labels in the classical sense
#train = torch.utils.data.TensorDataset(torch.Tensor(train).float())
#test = torch.utils.data.TensorDataset(torch.Tensor(test).float())
# Option 2: Custom DataSet
train = gjnn.dataloader.Dataset(train) 
test = gjnn.dataloader.Dataset(test)

In [8]:
batch_size = 64
n_iters = 500
num_epochs = n_iters / (len(train) / batch_size)
num_epochs = int(num_epochs)
print("The number of epochs is: " + str(num_epochs))

The number of epochs is: 33


In [9]:


train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)





print("After data loading")

# Setting other neural network hyperparameters
hidden_layer_size = 20
siamese_layer_size = 20
output_layer_size = 1
num_features_per_branch = 13
lr = 0.01
momentum = 0.9
num_epoch = 5



# Check dimensions of features
model = gjnn.model.SiameseNetwork(num_features_per_branch, siamese_layer_size, hidden_layer_size, output_layer_size)
print("Model correctly initialized...")

# Initialization of the Loss Function
criterion = gjnn.loss.DistanceLoss()
print("Distance Loss Correctly Initialized...")

# At the moment we stick to a classic SGD algorithm, maybe we can change it to Adam
#optimizer = torch.optim.Adam(model.parameters(), lr=lr)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
print("Optimizer Instantiated...")


iter = 0

# TEST ON A BATCH OF THE DATASET
for i, (user1, user2, user1_dist, user2_dist) in enumerate(train_loader):
    print(i)
    print(user1, user2, user1_dist, user2_dist)

After data loading
Model correctly initialized...
Distance Loss Correctly Initialized...
Optimizer Instantiated...
0
tensor([[   0.1751,    0.0000,    0.6768,    0.1435,    0.0000,    0.0000,
            2.0000,  600.0000,    0.1000,    0.9000,    0.0000,    0.0000,
            0.1414],
        [   0.1751,    0.0000,    0.6768,    0.1435,    0.0000,    0.0000,
            2.0000,  600.0000,    0.1000,    0.9000,    0.0000,    0.0000,
            0.1414],
        [   0.1751,    0.0000,    0.6768,    0.1435,    0.0000,    0.0000,
            2.0000,  600.0000,    0.1000,    0.9000,    0.0000,    0.0000,
            0.1414],
        [   0.1751,    0.0000,    0.6768,    0.1435,    0.0000,    0.0000,
            2.0000,  600.0000,    0.1000,    0.9000,    0.0000,    0.0000,
            0.1414],
        [   0.1751,    0.0000,    0.6768,    0.1435,    0.0000,    0.0000,
            2.0000,  600.0000,    0.1000,    0.9000,    0.0000,    0.0000,
            0.1414],
        [   0.1751,    0.000

tensor([[   0.1751,    0.0000,    0.6768,    0.1435,    0.0000,    0.0000,
            2.0000,  600.0000,    0.1000,    0.9000,    0.0000,    0.0000,
            0.1414],
        [   0.1751,    0.0000,    0.6768,    0.1435,    0.0000,    0.0000,
            2.0000,  600.0000,    0.1000,    0.9000,    0.0000,    0.0000,
            0.1414],
        [   0.1751,    0.0000,    0.6768,    0.1435,    0.0000,    0.0000,
            2.0000,  600.0000,    0.1000,    0.9000,    0.0000,    0.0000,
            0.1414],
        [   0.1751,    0.0000,    0.6768,    0.1435,    0.0000,    0.0000,
            2.0000,  600.0000,    0.1000,    0.9000,    0.0000,    0.0000,
            0.1414],
        [   0.1751,    0.0000,    0.6768,    0.1435,    0.0000,    0.0000,
            2.0000,  600.0000,    0.1000,    0.9000,    0.0000,    0.0000,
            0.1414],
        [   0.1751,    0.0000,    0.6768,    0.1435,    0.0000,    0.0000,
            2.0000,  600.0000,    0.1000,    0.9000,    0.0000,    0.0

8
tensor([[   0.1751,    0.0000,    0.6768,    0.1435,    0.0000,    0.0000,
            2.0000,  600.0000,    0.1000,    0.9000,    0.0000,    0.0000,
            0.1414],
        [   0.1751,    0.0000,    0.6768,    0.1435,    0.0000,    0.0000,
            2.0000,  600.0000,    0.1000,    0.9000,    0.0000,    0.0000,
            0.1414],
        [   0.1751,    0.0000,    0.6768,    0.1435,    0.0000,    0.0000,
            2.0000,  600.0000,    0.1000,    0.9000,    0.0000,    0.0000,
            0.1414],
        [   0.1751,    0.0000,    0.6768,    0.1435,    0.0000,    0.0000,
            2.0000,  600.0000,    0.1000,    0.9000,    0.0000,    0.0000,
            0.1414],
        [   0.1751,    0.0000,    0.6768,    0.1435,    0.0000,    0.0000,
            2.0000,  600.0000,    0.1000,    0.9000,    0.0000,    0.0000,
            0.1414],
        [   0.1751,    0.0000,    0.6768,    0.1435,    0.0000,    0.0000,
            2.0000,  600.0000,    0.1000,    0.9000,    0.0000,    0

13
tensor([[   0.1751,    0.0000,    0.6768,    0.1435,    0.0000,    0.0000,
            2.0000,  600.0000,    0.1000,    0.9000,    0.0000,    0.0000,
            0.1414],
        [   0.1751,    0.0000,    0.6768,    0.1435,    0.0000,    0.0000,
            2.0000,  600.0000,    0.1000,    0.9000,    0.0000,    0.0000,
            0.1414],
        [   0.1751,    0.0000,    0.6768,    0.1435,    0.0000,    0.0000,
            2.0000,  600.0000,    0.1000,    0.9000,    0.0000,    0.0000,
            0.1414],
        [   0.1751,    0.0000,    0.6768,    0.1435,    0.0000,    0.0000,
            2.0000,  600.0000,    0.1000,    0.9000,    0.0000,    0.0000,
            0.1414],
        [   0.1751,    0.0000,    0.6768,    0.1435,    0.0000,    0.0000,
            2.0000,  600.0000,    0.1000,    0.9000,    0.0000,    0.0000,
            0.1414],
        [   0.1751,    0.0000,    0.6768,    0.1435,    0.0000,    0.0000,
            2.0000,  600.0000,    0.1000,    0.9000,    0.0000,    

In [11]:
for epoch in range(num_epochs):
    print("Epoch " + str(epoch))
    print(train_loader)
    for i, (user_1, user_2, user_1_dist, user_2_dist) in enumerate(train_loader):
        #features_u1 = Variable(user_1.view(-1, num_features))
        #features_u2 = Variable(user_2.view(-1, num_features))
        features_u1 = Variable(user_1)
        features_u2 = Variable(user_2)
        dist_u1 = Variable(user_1_dist)
        dist_u2 = Variable(user_2_dist)
        
        optimizer.zero_grad()
        
        # Here we have to give data which goes to branch 1 and data who goes on branch 2
        outputs = model(features_u1, features_u2)
        
        loss = criterion(user_1_dist, user_2_dist, outputs)
        print("loss for i {} is equal to: {}".format(i, loss))
        
        loss.backward()
        
        optimizer.step()
        
        iter += 1
        print(iter)
        # we want to check the accuracy with test dataset every 500 iterations
        # we can change this number, it is just if it is too small we lose a lot of time
        # checking accuracy while if it is big, we have less answers but takes less time for the algorithm
        #if iter % 500 == 0:
            # calculate accuracy
        #    correct = 0
        #    total = 0
            
            # iterate through test dataset
         #   for features, labels in test_loader:
         #       features = Variable(features.view(-1, num_features))
                
         #      outputs = model(features)
                # get predictions from the maximum value
         #       _, predicted = torch.max(outputs.data, 1)
                
                # total number of labels
         #       total += labels.size(0)
                
         #       correct += (predicted == labels).sum()
            
         #   accuracy = 100 * correct / total
            
         #   print("Iteration: {}. Loss: {}. Accuracy: {}".format(iter, loss.data[0], accuracy))

Epoch 0
loss for i 0 is equal to: 6326.13525390625
496
loss for i 1 is equal to: 6317.44482421875
497
loss for i 2 is equal to: 6309.89111328125
498
loss for i 3 is equal to: 6340.43994140625
499
loss for i 4 is equal to: 6299.16455078125
500
loss for i 5 is equal to: 6303.8251953125
501
loss for i 6 is equal to: 6324.50048828125
502
loss for i 7 is equal to: 6322.0556640625
503
loss for i 8 is equal to: 6316.71826171875
504
loss for i 9 is equal to: 6288.7333984375
505
loss for i 10 is equal to: 6339.853515625
506
loss for i 11 is equal to: 6308.9345703125
507
loss for i 12 is equal to: 6314.1630859375
508
loss for i 13 is equal to: 6315.62255859375
509
loss for i 14 is equal to: 5330.82373046875
510
Epoch 1
loss for i 0 is equal to: 6336.7880859375
511
loss for i 1 is equal to: 6331.9873046875
512
loss for i 2 is equal to: 6311.71240234375
513
loss for i 3 is equal to: 6304.75146484375
514
loss for i 4 is equal to: 6318.890625
515
loss for i 5 is equal to: 6314.1796875
516
loss for i

loss for i 12 is equal to: 6324.14892578125
658
loss for i 13 is equal to: 6353.46630859375
659
loss for i 14 is equal to: 5319.89990234375
660
Epoch 11
loss for i 0 is equal to: 6292.6884765625
661
loss for i 1 is equal to: 6319.27685546875
662
loss for i 2 is equal to: 6321.81787109375
663
loss for i 3 is equal to: 6323.140625
664
loss for i 4 is equal to: 6315.83154296875
665
loss for i 5 is equal to: 6310.3466796875
666
loss for i 6 is equal to: 6332.9931640625
667
loss for i 7 is equal to: 6317.49951171875
668
loss for i 8 is equal to: 6323.11669921875
669
loss for i 9 is equal to: 6325.44140625
670
loss for i 10 is equal to: 6319.9775390625
671
loss for i 11 is equal to: 6313.6181640625
672
loss for i 12 is equal to: 6303.16650390625
673
loss for i 13 is equal to: 6316.48046875
674
loss for i 14 is equal to: 5322.90966796875
675
Epoch 12
loss for i 0 is equal to: 6329.76318359375
676
loss for i 1 is equal to: 6316.8193359375
677
loss for i 2 is equal to: 6311.11767578125
678
loss

loss for i 10 is equal to: 6330.86083984375
821
loss for i 11 is equal to: 6315.87646484375
822
loss for i 12 is equal to: 6329.07861328125
823
loss for i 13 is equal to: 6290.53662109375
824
loss for i 14 is equal to: 5337.6123046875
825
Epoch 22
loss for i 0 is equal to: 6319.451171875
826
loss for i 1 is equal to: 6312.2822265625
827
loss for i 2 is equal to: 6331.82177734375
828
loss for i 3 is equal to: 6304.68701171875
829
loss for i 4 is equal to: 6329.6455078125
830
loss for i 5 is equal to: 6298.38037109375
831
loss for i 6 is equal to: 6333.10107421875
832
loss for i 7 is equal to: 6306.4384765625
833
loss for i 8 is equal to: 6322.97900390625
834
loss for i 9 is equal to: 6307.6611328125
835
loss for i 10 is equal to: 6309.85791015625
836
loss for i 11 is equal to: 6302.67333984375
837
loss for i 12 is equal to: 6310.49853515625
838
loss for i 13 is equal to: 6333.43017578125
839
loss for i 14 is equal to: 5335.3984375
840
Epoch 23
loss for i 0 is equal to: 6309.9228515625
8

loss for i 6 is equal to: 6327.6162109375
982
loss for i 7 is equal to: 6319.70849609375
983
loss for i 8 is equal to: 6307.44482421875
984
loss for i 9 is equal to: 6321.5185546875
985
loss for i 10 is equal to: 6304.75830078125
986
loss for i 11 is equal to: 6317.47265625
987
loss for i 12 is equal to: 6316.30419921875
988
loss for i 13 is equal to: 6324.3115234375
989
loss for i 14 is equal to: 5313.98388671875
990
